# Analisis Teks

NLP (Pemrosesan Bahasa Alami) adalah cabang dari kecerdasan buatan (AI) yang berhubungan dengan bahasa tertulis dan lisan. Anda dapat menggunakan NLP untuk membuat solusi yang mengekstrak arti semantik dari teks atau lisan, atau yang memformulasikan respons bermakna dalam bahasa alami.

*Layanan kognitif* Microsoft Azure mencakup layanan *Analisis Teks*, yang menyediakan beberapa kemampuan NLP yang tidak biasa, termasuk identifikasi frasa kunci dalam teks, dan klasifikasi teks berdasarkan sentimen.

![Robot membaca buku catatan](./images/NLP.jpg)

Misalnya, organisasi fiktif *Margie's Travel* mendorong pelanggan untuk mengirim ulasan tentang menginap di hotel. Anda dapat menggunakan layanan Analisis Teks untuk membuat ringkasan ulasan dengan mengekstrak frasa kunci, menentukan mana ulasan positif dan negatif, atau menganalisis teks ulasan untuk sebutan atas entitas yang diketahui seperti lokasi atau orang.

## Melihat Dokumen Ulasan

Mari mulai dengan memperhatikan beberapa ulasan hotel yang diberikan oleh pelanggan.

Ulasan berada dalam file teks. Untuk melihatnya, cukup jalankan kode di bawah dengan mengeklik tombol **Jalankan sel** (&#9655;) di sebelah kiri sel.

In [ ]:
import os

# Read the reviews in the /data/reviews folder
reviews_folder = os.path.join('data', 'text', 'reviews')

# Create a collection of reviews with id (file name) and text (contents) properties
reviews = []
for file_name in os.listdir(reviews_folder):
    review_text = open(os.path.join(reviews_folder, file_name)).read()
    review = {"id": file_name, "text": review_text}
    reviews.append(review)

for review_num in range(len(reviews)):
    # print the review text
    print('{}\n{}\n'.format(reviews[review_num]['id'], reviews[review_num]['text']))

## Membuat Sumber Daya Cognitive Services

Untuk menganalisis teks dalam ulasan ini, Anda dapat menggunakan layanan kognitif **Analisis Teks**. Untuk menggunakannya, Anda perlu menyediakan sumber daya **Analisis Teks** atau **Cognitive Services** di langganan Azure (Gunakan sumber daya Analisis Teks jika ini satu-satunya layanan yang Anda rencanakan untuk digunakan atau Anda ingin melacak penggunaannya secara terpisah; atau Anda dapat menggunakan sumber daya Cognitive Services untuk menggabungkan layanan Analisis Teks dengan layanan kognitif lain, memungkinkan pengembang menggunakan kunci dan titik akhir tunggal untuk mengaksesnya.)

Jika Anda belum memilikinya, gunakan langkah-langkah berikut untuk membuat sumber daya **Cognitive Services** di langganan Azure Anda:

> **Catatan**: Jika Anda sudah memiliki sumber daya Cognitive Services, cukup buka halaman Mulai cepat di portal Microsoft Azure dan salin kunci dan titik akhirnya ke sel di bawah. Atau, ikuti langkah-langkah di bawah untuk membuatnya.

1. Di tab browser lain, buka portal Microsoft Azure di https://portal.azure.com, masuk menggunakan akun Microsoft Anda.
2. Klik tombol **&#65291;Buat sumber daya**, cari *Cognitive Services*, dan buat sumber daya **Cognitive Services** dengan pengaturan berikut:
    - **Langganan**: *Langganan Azure Anda*.
    - **Grup sumber daya**: *Pilih atau buat grup sumber daya dengan nama unik*.
    - **Wilayah**: *Pilih wilayah yang tersedia*:
    - **Nama**: *Masukkan nama yang unik*.
    - **Tingkat Harga**: S0
    - **Saya mengonfirmasi bahwa saya telah membaca dan memahami pemberitahuan tersebut**: Dipilih.
3. Tunggu penyebaran hingga selesai. Lalu, buka sumber daya layanan kognitif, dan di halaman **Ringkasan**, klik tautan untuk mengelola kunci layanan. Anda akan memerlukan titik akhir dan kunci untuk terhubung ke sumber daya layanan kognitif Anda dari aplikasi klien.

### Mendapatkan Kunci dan Titik Akhir untuk sumber daya Cognitive Services Anda

Untuk menggunakan sumber daya layanan kognitif, aplikasi klien memerlukan titik akhir dan kunci autentikasi:

1. Di portal Azure, di halaman **Kunci dan Titik Akhir** untuk sumber daya layanan kognitif Anda, salin **Kunci1** untuk sumber daya dan tempel pada kode di bawah, menggantikan **YOUR_COG_KEY**.
2. Salin **titik akhir** untuk sumber daya dan tempel pada kode di bawah, menggantikan **YOUR_COG_ENDPOINT**.
3. Jalankan kode pada sel di bawah dengan mengeklik tombol hijau <span style="color:green">&#9655;</span>.

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

## Mendeteksi Bahasa
Mari mulai dengan mengidentifikasi bahasa ulasan ini.

In [ ]:
import os
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials

# Get a client for your text analytics cognitive service resource
text_analytics_client = TextAnalyticsClient(endpoint=cog_endpoint,
                                            credentials=CognitiveServicesCredentials(cog_key))

# Analyze the reviews you read from the /data/reviews folder earlier
language_analysis = text_analytics_client.detect_language(documents=reviews)

# print detected language details for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the language details for this review
    lang = language_analysis.documents[review_num].detected_languages[0]
    print(' - Language: {}\n - Code: {}\n - Score: {}\n'.format(lang.name, lang.iso6391_name, lang.score))

    # Add the detected language code to the collection of reviews (so we can do further analysis)
    reviews[review_num]["language"] = lang.iso6391_name

## Mengekstrak Frasa Kunci

Sekarang Anda dapat menganalisis teks dalam ulasan pelanggan untuk mengidentifikasi frasa kunci yang memberikan beberapa indikasi poin pembicaraan utama.

In [ ]:
# # Use the client and reviews you created in the previous code cell to get key phrases
key_phrase_analysis = text_analytics_client.key_phrases(documents=reviews)

# print key phrases for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the key phrases in this review
    print('\nKey Phrases:')
    key_phrases = key_phrase_analysis.documents[review_num].key_phrases
    # Print each key phrase
    for key_phrase in key_phrases:
        print('\t', key_phrase)
    print('\n')

Frasa kunci dapat membantu Anda mendapatkan pemahaman tentang poin pembicaraan paling penting dalam setiap ulasan. Misalnya, sebuah ulasan berisi frasa "staf yang sangat membantu" atau "layanan yang buruk" dapat memberikan indikasi beberapa keprihatinan utama dari pengulas kepada Anda.

## Menentukan Sentimen

Mungkin berguna untuk mengklasifikasikan ulasan sebagai *positif* atau *negatif* berdasarkan *skor sentimen*. Sekali lagi, Anda dapat menggunakan layanan Analisis Teks untuk melakukan ini.

In [ ]:
# Use the client and reviews you created previously to get sentiment scores
sentiment_analysis = text_analytics_client.sentiment(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):

    # Get the sentiment score for this review
    sentiment_score = sentiment_analysis.documents[review_num].score

    # classifiy 'positive' if more than 0.5, 
    if sentiment_score < 0.5:
        sentiment = 'negative'
    else:
        sentiment = 'positive'

    # print file name and sentiment
    print('{} : {} ({})'.format(reviews[review_num]['id'], sentiment, sentiment_score))

## Mengekstrak Entitas yang Telah Diketahui

*Entitas* adalah hal yang bisa jadi disebutkan dalam teks yang merujuk pada beberapa jenis item yang secara umum telah dipahami. Misalnya, lokasi, orang, atau tanggal. Misalnya, Anda tertarik pada tanggal dan tempat yang disebut dalam ulasan, Anda dapat menggunakan kode berikut untuk mencarinya.

In [ ]:
# Use the client and reviews you created previously to get named entities
entity_analysis = text_analytics_client.entities(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):
    print(reviews[review_num]['id'])
    # Get the named entitites in this review
    entities = entity_analysis.documents[review_num].entities
    for entity in entities:
        # Only print datetime or location entitites
        if entity.type in ['DateTime','Location']:
            link = '(' + entity.wikipedia_url + ')' if entity.wikipedia_id is not None else ''
            print(' - {}: {} {}'.format(entity.type, entity.name, link))

Perhatikan bahwa beberapa entitas cukup dikenal baik memiliki kaitan dengan halaman Wikipedia, dalam hal ini layanan Analisis Teks menampilkan URL untuk halaman tersebut.

## Pelajari selengkapnya

Untuk informasi selengkapnya tentang layanan Analisis Teks, lihat [dokumentasi layanan Analisis Teks](https://docs.microsoft.com/azure/cognitive-services/text-analytics/)